In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from pyspark.sql.functions import isnan, when, count, col

from google.cloud import storage
from functools import reduce

In [2]:
spark = SparkSession.builder \
  .appName('DexcomCleaning') \
  .getOrCreate()

In [3]:
#Set output configuration
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [4]:
BUCKET_NAME = "maude-device-reports"

In [ ]:
#View datasets in blob storage
gcs_client = storage.Client()
bucket = gcs_client.bucket(BUCKET_NAME)

list(bucket.list_blobs())

In [5]:
def load_text_data(year):
    
    text_schema = StructType([
        StructField("MDR_REPORT_KEY", IntegerType()),
        StructField("MDR_TEXT_KEY", IntegerType()),
        StructField("TEXT_TYPE_CODE", StringType()),
        StructField("PATIENT_SEQUENCE_NUMBER", IntegerType()),
        StructField("DATE_REPORT", DateType()),
        StructField("FOI_TEXT", StringType())
    ])
    
    
    text_df = spark \
        .read \
        .schema(text_schema) \
        .option("header" , "true") \
        .option("delimiter", "|") \
        .csv(f"./foitext{year}.txt")
    
    # Replace to this if using in cloud environment: .csv(f"gs://{BUCKET_NAME}/foitext{year}.txt")
    
    return text_df

def load_device_data(year):

    device_schema = StructType([
        StructField("MDR_REPORT_KEY", IntegerType()),
        StructField("DEVICE_EVENT_KEY", IntegerType()),
        StructField("IMPLANT_FLAG", StringType()),
        StructField("DATE_REMOVED_FLAG", DateType()),
        StructField("DEVICE_SEQUENCE_NO", IntegerType()),
        StructField("DATE_RECEIVED", StringType()), #StructField("DATE_RECEIVED", DateType()),
        StructField("BRAND_NAME", StringType()),
        StructField("GENERIC_NAME", StringType()),
        StructField("MANUFACTURER_D_NAME", StringType()),
        StructField("MANUFACTURER_D_ADDRESS_1", StringType()),
        StructField("MANUFACTURER_D_ADDRESS_2", StringType()),
        StructField("MANUFACTURER_D_CITY", StringType()),
        StructField("MANUFACTURER_D_STATE_CODE", StringType()),
        StructField("MANUFACTURER_D_ZIP_CODE", StringType()),
        StructField("MANUFACTURER_D_ZIP_CODE_EXT", StringType()),
        StructField("MANUFACTURER_D_COUNTRY_CODE", StringType()),
        StructField("MANUFACTURER_D_POSTAL_CODE", StringType()),
        StructField("DEVICE_OPERATOR", StringType()),
        StructField("EXPIRATION_DATE_OF_DEVICE", DateType()),
        StructField("MODEL_NUMBER", IntegerType()),
        StructField("CATALOG_NUMBER", IntegerType()),
        StructField("LOT_NUMBER", IntegerType()),
        StructField("OTHER_ID_NUMBER", StringType()),
        StructField("DEVICE_AVAILABILITY", StringType()),
        StructField("DATE_RETURNED_TO_MANUFACTURER", DateType()),
        StructField("DEVICE_REPORT_PRODUCT_CODE", StringType()),
        StructField("DEVICE_AGE_TEXT", StringType()),
        StructField("DEVICE_EVALUATED_BY_MANUFACTUR", StringType()),
        StructField("COMBINATION_PRODUCT_FLAG", StringType())
    ])
    
    
    device_df = spark \
        .read \
        .schema(device_schema) \
        .option("header" , "true") \
        .option("delimiter", "|") \
        .csv(f"./device{year}.txt")
    
    ## Replace to this if using in cloud environment: .csv(f"gs://{BUCKET_NAME}/device{year}.txt")
    
    return device_df

In [6]:
text16 = load_text_data(2016)
text17 = load_text_data(2017)
text18 = load_text_data(2018)
text19 = load_text_data(2019)
text20 = load_text_data(2020)
text21 = load_text_data(2021)

In [7]:
device16 = load_device_data(2016)
device17 = load_device_data(2017)
device18 = load_device_data(2018)
device19 = load_device_data(2019)
device20 = load_device_data(2020)
device21 = load_device_data(2021)

In [9]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [10]:
text_all = unionAll(text16, text17, text18, text19, text20, text21) #16802162 records
device_all = unionAll(device16, device17, device18, device19, device20, device21) #7790750 records

In [11]:
#Create Temp tables for device and text
text_all.createOrReplaceTempView("text_tbl")
device_all.createOrReplaceTempView("device_tbl")

In [23]:
dexcom_reports_query = """
    SELECT 
        device.MDR_REPORT_KEY,
        to_date(device.DATE_RECEIVED,'yyyy/mm/dd') as DATE_RECEIVED,
        device.BRAND_NAME,
        device.GENERIC_NAME,
        device.MANUFACTURER_D_NAME,
        text.FOI_TEXT
    FROM text_tbl AS text 
    LEFT JOIN device_tbl AS device 
    ON text.MDR_REPORT_KEY ==  device.MDR_REPORT_KEY
    WHERE text.TEXT_TYPE_CODE = 'D' AND device.BRAND_NAME LIKE '%DEXCOM%'
    """

In [24]:
dexcom_df = spark.sql(dexcom_reports_query)

In [30]:
#Export one file locally
dexcom_df.coalesce(1).write.csv("dexcom_reports_all")

In [ ]:
#Export one file to bucket
#all_filepath = f"gs://{BUCKET_NAME}/reports_all.csv"
dexcom_filepath = f"gs://{BUCKET_NAME}/dexcom_reports_all.csv"

#reports_all.write.mode('overwrite').csv(all_filepath)
dexcom_df.coalesce(1).write.mode('overwrite').csv(dexcom_filepath)

In [38]:
spark.stop()